# 卷积
两个原则：平移不变性，局部性<br>
## 重新考察全连接层
需要将权重变为4-D张量: $$h_{i,j} = \sum _{k,l} w_{i,j,k,l} x_{k,l} = \sum _{a,b} v_{i,j,a,b} x_{i+a,j+b}$$
V是W的重新索引 $v_{i,j,a,b} = w_{i,j,i+a,j+b}$

### 原则1 平移不变性
希望做到x平移后，h不应当平移
因为在式 $$h_{i,j} = \sum _{a,b} v_{i,j,a,b} x_{i+a,j+b}$$
中，x平移会导致h的平移因此解决方案是：$v_{i,j,a,b} = v_{a,b}$ <br>
$$h_{i,j} = \sum _{a,b} v_{a,b} x_{i+a,j+b}$$
这就是二维卷积（交叉相关）


### 原则2 局部性
$$h_{i,j} = \sum _{a,b} v_{a,b} x_{i+a,j+b}$$
解决方案: $$ h_{i,j} = \sum _{a = - \Delta}^\Delta \sum _{b = - \Delta}^\Delta v_{a,b} x_{i+a,j+b}$$
这便是将全连接层转变为了卷积层的过程


二维交叉相关（对应项相乘后相加）<br>
输入X：$n_h * n_w$<br>
核W：$k_h * k_w $<br>
偏差b<br>
输出Y：$(n_h - k_h + 1) * (n_w - k_w + 1)$<br>
$$Y = X*W +b$$
W和b都是可学习的参数


#### 互相关运算实现（卷积）

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

def corr2d(X, K):  # X输入，K核矩阵
    """计算二维互相关运算。"""
    h, w = K.shape # 行数 列数
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum() # * .sum()实现点乘运算
    return Y

In [2]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

实现二维卷积层

In [3]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size)) # 将这个卷积核设置为可训练的参数
        self.bias = nn.Parameter(torch.zeros(1)) # 广播机制或许可以将这个偏移置于全局

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

卷积层应用：检测图像边缘

In [4]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X # 白变黑黑变白的两根线

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [5]:
K = torch.tensor([[1.0, -1.0]]) # 边缘检测
K.size()

torch.Size([1, 2])

In [6]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

该卷积核只能检测竖向的边

In [7]:
corr2d(X.t(), K) # 转置后的矩阵无法检测

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [8]:
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)

X = X.reshape((1, 1, 6, 8)) # 批量大小数 通道数 图片的两个维度
Y = Y.reshape((1, 1, 6, 7))

for i in range(10):
    Y_hat = conv2d(X) # 设置预测值（正向传播）
    l = (Y_hat - Y)**2 # 损失函数
    conv2d.zero_grad() # 梯度清零
    l.sum().backward() # 计算梯度 计算得到的梯度会被存储至对应张量的.grad中
    conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad # 梯度下降 学习率*梯度
    if (i + 1) % 2 == 0:
        print(f'batch {i+1}, loss {l.sum():.3f}')

batch 2, loss 12.647
batch 4, loss 2.645
batch 6, loss 0.658
batch 8, loss 0.198
batch 10, loss 0.069


所学习的卷积层权重

In [9]:
conv2d.weight.data.reshape((1, 2)) # 结果接近 1.-1

tensor([[ 1.0087, -0.9577]])

解决抖动：1.增大批量大小 2.增大学习率

## 填充和步幅
### 填充
在输入周围添加额外的行和列<br>
填充$p_h$行和$p_w$列，输出形状为$$(n_h - k_h + p_h + 1) * (n_w - k_w +p_w + 1)$$
通常取$p_h = k_h - 1, p_w = k_w - 1$这样输入输出大小不会发生变化<br>
$k_h$为偶数时填充$p_h / 2$
### 步幅
控制每次不止移动一格
给定高度$s_h$和宽度$s_w$的步幅，输出形状是$$(n_h - k_h + p_h + s_h) / s_h * (n_w - k_w +p_w + s_w) / s_w$$
假设$p_h = k_h - 1, p_w = k_w - 1$
$$(n_h + s_h - 1) / s_h * (n_w + s_w - 1) / s_w$$
假设输入高度和宽度可以被步幅整除
$$(n_h / s_h) * (n_w / s_w)$$

In [10]:
import torch
from torch import nn

def comp_conv2d(conv2d, X):
    X = X.reshape((1, 1) + X.shape) # 在维度前加入批量大小和通道数
    Y = conv2d(X)
    return Y.reshape(Y.shape[2:]) # 只返回最后两个维度即矩阵大小，前两个维度是批量大小和通道数

conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1) # padding =1上下左右都填充一行
X = torch.rand(8, 8)
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

In [11]:
conv2d = nn.Conv2d(1, 1, kernel_size=(5, 3), padding=(2, 1)) # padding = (2,1) 上下填充两行，左右填充一列
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

In [12]:
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape # 根据公式计算出形状

torch.Size([4, 4])

In [13]:
conv2d = nn.Conv2d(1, 1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4)) 
comp_conv2d(conv2d, X).shape # 根据公式计算出形状,向下取整

torch.Size([2, 2])

## 多个输入和输出通道
### 多个输入通道
彩色图片有RGB三个通道<br>
每个通道都有一个卷积核，结果是所有通道的卷积结果的和<br>
单输出通道<br>
输入X: $c_i \times n_h \times n_w$<br>
核W:$ c_i \times k_h \times k_w$<br>
输出Y: $ m_h \times m_w$<br>
$$Y = \sum _{i=0}^{c_i} X_{i,:,:} \star W_{i,:,:}$$
其中$\star$意为卷积操作

### 多输出通道
输入X: $c_i \times n_h \times n_w$<br>
核W:$ c_o \times c_i \times k_h \times k_w$<br>
输出Y: $ c_o \times m_h \times m_w$<br>
$$Y_{i,:,:} = X \star W_{i,:,:,:} \quad for \: i=1,...,c_o$$
每个输出通道可以识别为特定模式

### 1 * 1卷积层
不识别空间模式，仅仅融合通道，相当于输入形状为$n_h n_w \times c_i$ 经过一个形状为$c_o \times c_i$ 输出为$n_h n_w \times c_o$

### 二维卷积层
输入X: $c_i \times n_h \times n_w$<br>
核W: $c_o \times c_i \times k_h \times k_w$ 共有$c_o \times c_i$个核<br>
偏差B: $c_o \times c_i$ 每个核一个<br>
输出Y: $ c_o \times m_h \times m_w$<br>
$$Y = X \star W + B$$

In [14]:
import torch
from d2l import torch as d2l

def corr2d_multi_in(X, K): # 多输入通道\
    return sum(corr2d(x, k) for x, k in zip(X, K)) # zip将两个列表打包成一个元组列表
# 实际是拿出每个通道的x,k分别计算后相加
# output = torch.sum(torch.stack([torch.conv2d(X[i], K[i]) for i in range(C_in)]), dim=0)
# 该代码相当于，先在stack中拼接后再在第一维度即新增的维度相加，因为torch.sum只能对张量运算
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
                  [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])

K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

X.shape, K.shape, corr2d_multi_in(X, K).shape ,corr2d_multi_in(X, K)



(torch.Size([2, 3, 3]),
 torch.Size([2, 2, 2]),
 torch.Size([2, 2]),
 tensor([[ 56.,  72.],
         [104., 120.]]))

In [15]:
for x,k in zip(X, K):
    print(x.shape, k.shape)
    print(x, k)


torch.Size([3, 3]) torch.Size([2, 2])
tensor([[0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.]]) tensor([[0., 1.],
        [2., 3.]])
torch.Size([3, 3]) torch.Size([2, 2])
tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]]) tensor([[1., 2.],
        [3., 4.]])


计算多个通道的输出的互相关函数

In [16]:
def corr2d_multi_in_out(X, K): # 多输入多输出通道 
    return torch.stack([corr2d_multi_in(X, k) for k in K]) # 对每个k进行卷积

# 每个 for都是一次降维的过程
K = torch.stack((K, K + 1, K + 2))
K.shape

torch.Size([3, 2, 2, 2])

In [17]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

验证1*1 卷积等价于全连接

In [18]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6
Y1.shape

torch.Size([2, 3, 3])

## 池化层
卷积层对位置非常的敏感，因此我们希望将他更好的泛化，当图片发生一定程度的变化时，输出不要发生太大的变化，类似需要一点平移不变性<br>

### 二维最大池化
返回滑动窗口的最大值（卷积窗口的最大值）<br>
池化层与卷积层类似，都具有填充和步幅<br>
没有可学习的参数<br>
在每个输入通道应用池化层获得相应的输出通道<br>
输出通道数 = 输出通道数 （注意是通道数，不是形状）<br>

### 平均池化层
平均的池化

### 总结
池化层返回窗口中的最大或平均值<br>
缓解卷积层会位置敏感性<br>
同样有窗口大小、填充和步幅作为参数<br>

### 实现池化层

In [19]:
import torch
from torch import nn
from d2l import torch as d2l

def pool2d(X, pool_size, mode='max'): # 池化函数 mode就是最大还是平均
    p_h, p_w = pool_size
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i:i + p_h, j:j + p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i:i + p_h, j:j + p_w].mean()
    return Y

X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
pool2d(X, (2, 2)),pool2d(X, (2, 2), 'avg')

(tensor([[4., 5.],
         [7., 8.]]),
 tensor([[2., 3.],
         [5., 6.]]))

### 填充和步幅

In [20]:
X = torch.arange(16, dtype=torch.float32).reshape((1, 1, 4, 4))
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

In [21]:
pool2d = nn.MaxPool2d(3) # pytorch框架中似乎规定了窗口不重叠
pool2d(X)

tensor([[[[10.]]]])

填充和步幅可以手动设定

In [22]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

In [23]:
pool2d = nn.MaxPool2d((2, 3), padding=(1, 1), stride=(2, 3))
pool2d(X)

tensor([[[[ 1.,  3.],
          [ 9., 11.],
          [13., 15.]]]])

池化层在每个输入通道上单独运算

In [24]:
X = torch.cat((X, X + 1), 1)
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

In [25]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])